In [ ]:
# %pip install bs4
# %pip install pypandoc
# %pip install lxml
# %pip install striprtf

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [1]:

from striprtf.striprtf import rtf_to_text
import pandas as pd
import regex as re
import matplotlib.pyplot as plt
from plotnine import ggplot, aes, geom_line, labs, theme_minimal
import os
from bs4 import BeautifulSoup
import pypandoc
import pandas as pd
from lxml import etree
# Download and install pandoc
pypandoc.download_pandoc()

In [2]:
# Load and read the RTF file
file_paths = [f"Factiva Data/Factiva-20250102-{i}.rtf" for i in range(1, 65)] 
rtf_contents = []

for file_path in file_paths:
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            rtf_contents.append(file.read())
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        continue

# Convert RTF content to plain text
plain_texts = [rtf_to_text(rtf_content) for rtf_content in rtf_contents]

# Split the plain text into individual articles (improve splitting logic if needed)
articles = []
for plain_text in plain_texts:
    articles.extend([article for article in plain_text.split('\n') if article.strip()])

In [4]:
# Initialize an empty list to store the separated articles
separated_articles = []
current_article = []

# Iterate through each line in the articles list
for line in articles:
    current_article.append(line)
    if re.search(r'Document \w+', line):
        separated_articles.append('\n'.join(current_article))
        current_article = []

# Add any remaining article that hasn't been added
if current_article:
    separated_articles.append('\n'.join(current_article))

# Replace the placeholder with the separated articles
articles = separated_articles

In [5]:
# Define the pattern to match the lines to be removed
pattern = re.compile(r"Search Summary.*?Timestamp\|\d{1,2} \w+ \d{4} \d{2}:\d{2}", re.DOTALL)

# Filter the articles
filtered_articles = [pattern.sub('', article).strip() for article in articles]

# Replace the placeholder with the filtered articles
articles = filtered_articles

In [6]:
# Extract relevant fields from each article
data = []
for article in articles:
    lines = article.split('\n')
    try:
        # headline
        headline = next(line for line in lines if re.search(r'\b\d+\s+words\b', line))
        headline_index = lines.index(headline)
        headline = '\n'.join(lines[:headline_index]).strip()
        #num_words
        num_words = next(line for line in lines if re.search(r'\b\d+\s+words\b', line))
        num_words_index = lines.index(num_words)
        #date
        date = next(line for line in lines[num_words_index:] if re.search(r'\b\d{1,2} \w+ \d{4}\b', line))
        date_index = lines.index(date)
        publisher = lines[date_index + 1]
        publisher_code_index = lines.index(publisher) + 1
        publisher_code = next(line for line in lines[publisher_code_index:] if re.search(r'\b[A-Z]+\b', line))
        language = next(line for line in lines[publisher_code_index + 1:] if re.search(r'\b[A-Za-z]+\b', line))
        # copyright_info = next(line for line in lines if re.search(r'\bCopyright\b', line))
        copyright_info = lines[lines.index(language) + 1]
        main_article = ' '.join(lines[lines.index(copyright_info) + 1:])
        main_article = re.split(r'Document \w+', main_article)[0].strip()
        
        data.append({
            'Headline': headline,
            'Number of Words': num_words,
            'Date': date,
            'Publisher': publisher,
            'Publisher Code': publisher_code,
            'Language': language,
            'Copyright': copyright_info,
            'Main Article': main_article
        })
    except (IndexError, StopIteration):
        continue

# Convert extracted data into a DataFrame
df = pd.DataFrame(data)


In [7]:
df

,Headline,Number of Words,Date,Publisher,Publisher Code,Language,Copyright,Main Article
0,TRADE MINISTERS OF APEC TO DISCUSS NEW GATT HERE.,585 words,4 October 1994,Jakarta Post,JKPOST,English,(c) 1994 The Jakarta Post,JAKARTA (JP): Ministers responsible for trade ...
1,GENERAL OUTLOOK FOR THE OIL MARKET. \n By Subroto,838 words,7 October 1994,Jakarta Post,JKPOST,English,(c) 1994 The Jakarta Post,The following is the first of a two-part artic...
2,GENERAL OUTLOOK FOR THE OIL MARKET (2).,"1,620 words",8 October 1994,Jakarta Post,JKPOST,English,(c) 1994 The Jakarta Post,The following is the second of a two-part arti...
3,TRADE LIBERALIZATION SOUGHT.,717 words,17 October 1994,Jakarta Post,JKPOST,English,(c) 1994 The Jakarta Post,JAKARTA (JP): A business advisory group of APE...
4,INFLATIONARY PRESSURES.,597 words,4 November 1994,Jakarta Post,JKPOST,English,(c) 1994 The Jakarta Post,"Over the last few months, President Soeharto h..."
...,...,...,...,...,...,...,...,...
6335,Business\nRupiah Gains Ground Amid Declining D...,504 words,30 December 2024,The Jakarta Globe,JGLOBE,English,Copyright 2024. PT Jakarta Globe Media.,Jakarta. The Indonesian rupiah closed stronger...
6336,Business\n Uncertainty Hits Crypto Industry A...,695 words,30 December 2024,The Jakarta Globe,JGLOBE,English,Copyright 2024. PT Jakarta Globe Media.,Jakarta. Uncertainty currently hits Indonesia’...
6337,IDX falters in 2024 as global markets soar to ...,800 words,31 December 2024,The Jakarta Post,JKPOST,English,(c) 2024 The Jakarta Post,A stellar year for stock markets around the wo...
6338,Business\nIndonesian Stock Market Faces Headwi...,302 words,31 December 2024,The Jakarta Globe,JGLOBE,English,Copyright 2024. PT Jakarta Globe Media.,Jakarta. The Indonesian stock market is set to...


In [8]:
df.to_csv('Factiva_Indo_News.csv', index=False)
